# Subsections Issue

## Setup

### Imports

In [1]:
from typing import List
from pathlib import Path
from pprint import pprint
import re
import sys

import pandas as pd
import xlwings as xw

from buffered_iterator import BufferedIterator
import text_reader as tp
from sections import Rule, RuleSet, SectionBreak, ProcessingMethods, Section

### Logging

In [2]:
import logging
logging.basicConfig(format='%(name)-20s - %(levelname)s: %(message)s')
#logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger('Text Processing')
#logger.setLevel(logging.DEBUG)
logger.setLevel(logging.INFO)

### Initialize 2-line Section Tests

#### 2-line Section Source

In [3]:
GENERIC_TEST_TEXT = [
    'Text to be ignored',
    'StartSection Name: A',
    'EndSection Name: A',
    'StartSection Name: B',
    'EndSection Name: B',
    'More text to be ignored',
    ]

## Simple sections experimenting with start and end settings

### Two line section *StartSection*
- Start *Before* `StartSection`
- End *After* `EndSection`

In [4]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After')
    )

### Iterator Options

- If a regular iterator is supplied the *Step Back* item is lost
- The second call to `pprint(sub_section.read(test_iter))` returns an empty list 
  because the second *StartSection* line is lost.

In [5]:
test_iter = iter(GENERIC_TEST_TEXT)
pprint(sub_section.read(test_iter))

pprint(sub_section.read(test_iter))

['StartSection Name: A', 'EndSection Name: A']
[]


- Supplying a *BufferedIterator* preserves the *Step Back* item.
- The Third call to `pprint(sub_section.read(test_iter))` returns an empty list 
  because the default *aggregate* method is `list`, so no items in the sequence 
  returns an empty list.

In [6]:
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section.read(test_iter))
pprint(sub_section.read(test_iter))
pprint(sub_section.read(test_iter))

['StartSection Name: A', 'EndSection Name: A']
['StartSection Name: B', 'EndSection Name: B']
[]


- Using `subsections=sub_section` avoids the need for `BufferedIterator(GENERIC_TEST_TEXT)`

In [7]:
full_section = Section(
    section_name='Full',
    subsections=sub_section,
    keep_partial=True
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A', 'EndSection Name: A'],
 ['StartSection Name: B', 'EndSection Name: B'],
 []]


![Good](../examples/Valid.png) The extra empty list at the end is expected here.
- The extra empty list at the end results from the last line `'More text to be ignored'`. 
- `full_section` includes this line, but it is not included in the subsection.  
- The `keep_partial=True` setting forces an empty set to be returned because no subsection data was found.

|Expected|Actual|
|-|-|
|`['StartSection Name: A', 'EndSection Name: A']`|`['StartSection Name: A', 'EndSection Name: A']`|
|`['StartSection Name: B', 'EndSection Name: B']`|`['StartSection Name: B', 'EndSection Name: B']`|
|`[]`|`[]`|


Using `keep_partial=False` to eliminate empty list.

In [8]:
full_section = Section(
    section_name='Full',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=[sub_section],
    keep_partial=False
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A', 'EndSection Name: A'],
 ['StartSection Name: B', 'EndSection Name: B']]


![Good](../examples/Valid.png) 
The `keep_partial=False` setting forces the iterator to exit when full subsections are not found.

|Expected|Actual|
|-|-|
|`['StartSection Name: A', 'EndSection Name: A']`|`['StartSection Name: A', 'EndSection Name: A']`|
|`['StartSection Name: B', 'EndSection Name: B']`|`['StartSection Name: B', 'EndSection Name: B']`|
|||


### Single line section *StartSection*
> - Start *Before* `StartSection`
> - End *Before* `EndSection`

In [9]:
sub_section1 = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='Before')
    )

test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section1.read(test_iter))
pprint(sub_section1.read(test_iter))
pprint(sub_section1.read(test_iter))

['StartSection Name: A']
['StartSection Name: B']
[]


In [10]:

full_section = Section(
    section_name='Full',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=[sub_section1],
    keep_partial=False  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A'], ['StartSection Name: B'], []]


# Is the extra blank line unexpected?

![Bad](../examples/error.png) Extra empty list at the end.

|Expected|Actual|
|-|-|
|`['StartSection Name: A']`|`['StartSection Name: A']`|
|`['StartSection Name: B']`|`['StartSection Name: B']`|
||`[]`|


### Single line section *EndSection*
#### Attempt #1
> - Start *Before* `EndSection`
> - End *Before* `StartSection`

- This doesn't work well because it includes the unwanted text after the last *EndSection*

In [11]:
sub_section2a = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before')
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2a.read(test_iter))
pprint(sub_section2a.read(test_iter))
pprint(sub_section2a.read(test_iter))

['EndSection Name: A']
['EndSection Name: B', 'More text to be ignored']
[]


![Good](../examples/Valid.png) 

|Expected|Actual|
|-|-|
|`['EndSection Name: A']`|`['EndSection Name: A']`|
|`['EndSection Name: B', 'More text to be ignored']`|`['EndSection Name: B', 'More text to be ignored']`|
|`[]`|`[]`|


#### Attempt #2
> - Start *Before* `EndSection`
> - End *After* `EndSection`

- This doesn't work because The first line is not tested so it only breaks after the second occurrence of *EndSection*.

In [12]:
sub_section2b = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2b.read(test_iter))
pprint(sub_section2b.read(test_iter))
pprint(sub_section2b.read(test_iter))

['EndSection Name: A', 'StartSection Name: B', 'EndSection Name: B']
[]
[]


![Good](../examples/Valid.png) 

|Expected|Actual|
|-|-|
|`['EndSection Name: A', 'StartSection Name: B', 'EndSection Name: B']`|`['EndSection Name: A', 'StartSection Name: B', 'EndSection Name: B']`|
|`[]`|`[]`|
|`[]`|`[]`|


#### Attempt #3
> - Start *Before* `EndSection`
> - End *After* `EndSection`
> - Enable testing of first item

- This should work because the first line should be tested and trigger a break.

In [13]:
sub_section2c = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    end_on_first_item=True,
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2c.read(test_iter))
pprint(sub_section2c.read(test_iter))
pprint(sub_section2c.read(test_iter))

['EndSection Name: A']
['EndSection Name: B']
[]


![Good](../examples/Valid.png) 

|Expected|Actual|
|-|-|
|`['EndSection Name: A']`|`['EndSection Name: A']`|
|`['EndSection Name: B']`|`['EndSection Name: B']`|
|`[]`|`[]`|


#### Attempt #3.5
> - Start *Before* `EndSection`
> - End *Before* `EndSection`
> - Enable testing of first item

- This should __<u>not</u>__ work because the first line should be tested and 
  trigger a break before returning anything.

In [14]:
sub_section2cc = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='Before'),
    end_on_first_item=True,
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2cc.read(test_iter))
pprint(sub_section2cc.read(test_iter))
pprint(sub_section2cc.read(test_iter))

[]
[]
[]


![Good](../examples/Valid.png) 

|Expected|Actual|
|-|-|
|`[]`|`[]`|
|`[]`|`[]`|
|`[]`|`[]`|


#### Attempt #4
> - Start *Before* `EndSection`
> - End is `True` (Always Break)
> - Enable testing of first item

- This should work because the first line should be tested and always trigger a break.

In [15]:
sub_section2d = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='After'),
    end_on_first_item=True,
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2d.read(test_iter))
pprint(sub_section2d.read(test_iter))
pprint(sub_section2d.read(test_iter))

['EndSection Name: A']
['EndSection Name: B']
[]


![Good](../examples/Valid.png) 

|Expected|Actual|
|-|-|
|`['StartSection Name: A']`|`['EndSection Name: A']`|
|`['StartSection Name: B']`|`['EndSection Name: B']`|
|`[]`|`[]`|

#### Attempt #5
> - Start *Before* `EndSection`
> - End is `True` (Always Break)
> - Don't enable testing of first item

- This should <u>not</u> work because the break should trigger on the second line.

In [16]:
sub_section2e = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='After')
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2e.read(test_iter))
pprint(sub_section2e.read(test_iter))
pprint(sub_section2e.read(test_iter))

['EndSection Name: A', 'StartSection Name: B']
['EndSection Name: B', 'More text to be ignored']
[]


![Good](../examples/Valid.png) 

|Expected|Actual|
|-|-|
|`['EndSection Name: A', 'StartSection Name: B']`|`['EndSection Name: A', 'StartSection Name: B']`|
|`['EndSection Name: B', 'More text to be ignored']`|`['EndSection Name: B', 'More text to be ignored']`|
|`[]`|`[]`|

**`end_section=SectionBreak(True)` defaults to `break_offset='Before'`**

#### Attempt #6
> - Start *Before* `EndSection`
> - End *After* ___`True`___ (Always Break)
> - Don't enable testing of first item

- This should <u>not</u> work because the break should trigger *After* the second line.

In [17]:
sub_section2f = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='After')
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2f.read(test_iter))
pprint(sub_section2f.read(test_iter))
pprint(sub_section2f.read(test_iter))

['EndSection Name: A', 'StartSection Name: B']
['EndSection Name: B', 'More text to be ignored']
[]


![Good](../examples/Valid.png)
|Expected|Actual|
|-|-|
|`['EndSection Name: A', 'StartSection Name: B']`|`['EndSection Name: A', 'StartSection Name: B']`|
|`['EndSection Name: B', 'More text to be ignored']`|`['EndSection Name: B', 'More text to be ignored']`|
|`[]`|`[]`|

#### Attempt #7
> - Start *Before* `EndSection`
> - Enable testing of first item
> - No End setting

- This should start with the first *EndSection* line and not stop until the end of the iterator.

In [18]:
sub_section2g = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_on_first_item=True,
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2g.read(test_iter))
pprint(sub_section2g.read(test_iter))
pprint(sub_section2g.read(test_iter))

['EndSection Name: A',
 'StartSection Name: B',
 'EndSection Name: B',
 'More text to be ignored']
[]
[]


![Good](../examples/Valid.png)
|Expected|Actual|
|-|-|
|`['EndSection Name: A', ['StartSection Name: B', 'EndSection Name: B', 'More text to be ignored']`|`['EndSection Name: A', 'StartSection Name: B', 'EndSection Name: B', 'More text to be ignored']`|
|`[]`|`[]`|
|`[]`|`[]`|


#### Single line section *EndSection* __Final Design__.
> - Start *Before* `EndSection`
> - End *Before* ___`True`___ (Always Break)
> - Don't enable testing of first item

- This should work because the break should trigger *Before* the second line.

In [19]:
sub_section2 = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='Before')
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2.read(test_iter))
pprint(sub_section2.read(test_iter))
pprint(sub_section2.read(test_iter))

['EndSection Name: A']
['EndSection Name: B']
[]


![Good](../examples/Valid.png) End breaks *Before* second line. 

|Expected|Actual|
|-|-|
|`['EndSection Name: A']`|`['EndSection Name: A']`|
|`['EndSection Name: B']`|`['EndSection Name: B']`|
|`[]`|`[]`|

### Combined Start and End subsections Single line section 
***sub_section1***
> - Start *Before* `StartSection`
> - End *Before* `EndSection`

***sub_section2*** 
> - Start *Before* `EndSection`
> - End *Before* ___`True`___ (Always Break)
> - Don't enable testing of first item

`subsections=[sub_section1, sub_section2]`

In [20]:

full_section = Section(
    section_name='Full',
    subsections=[sub_section1, sub_section2]
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[[['StartSection Name: A'], ['EndSection Name: A']],
 [['StartSection Name: B'], ['EndSection Name: B']]]


![Good](../examples/Valid.png) Tree list levels with single item in deepest level andm two items in each of the other two levels.

|Expected|Actual|
|-|-|
|`[['StartSection Name: A'], ['EndSection Name: A']]`|`[['StartSection Name: A'], ['EndSection Name: A']]`|
|`[['StartSection Name: B'], ['EndSection Name: B']]`|`[['StartSection Name: B'], ['EndSection Name: B']]`|

## Hysteresis Bug Fixed  ![Good](../examples/Valid.png) 
When `full_section` contains 
`end_section=SectionBreak('ignored', break_offset='Before')`, repeated calls produce different effects.

#### Define `SubSection`: 
- Start *Before* `StartSection`
- End *After* `EndSection`

In [21]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before', name='SubSectionStart'),
    end_section=SectionBreak('EndSection', break_offset='After', name='SubSectionEnd')
    )

- Clear Hysteresis by running `full_section` without setting `end_section`

In [22]:
full_section = Section(
    section_name='Full',
    subsections=sub_section,
    keep_partial=False
    )
#a = full_section.read(GENERIC_TEST_TEXT)
pprint(full_section.read(GENERIC_TEST_TEXT))
pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A', 'EndSection Name: A'],
 ['StartSection Name: B', 'EndSection Name: B'],
 []]
[['StartSection Name: A', 'EndSection Name: A'],
 ['StartSection Name: B', 'EndSection Name: B'],
 []]


- Define `full_section` with 
`end_section=SectionBreak('ignored', break_offset='Before')`.

In [23]:
full_section = Section(
    section_name='Full',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=sub_section
    )

- First time it runs as expected

In [24]:
pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A', 'EndSection Name: A'],
 ['StartSection Name: B', 'EndSection Name: B']]


![Good](../examples/Valid.png) 
Both full subsections found. No empty lists afterwards.

|Expected|Actual|
|-|-|
|`['StartSection Name: A', 'EndSection Name: A']`|`['StartSection Name: A', 'EndSection Name: A']`|
|`['StartSection Name: B', 'EndSection Name: B']`|`['StartSection Name: B', 'EndSection Name: B']`|

- The second time, a line in the first subsection is missing.

In [25]:
pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A', 'EndSection Name: A'],
 ['StartSection Name: B', 'EndSection Name: B']]


![Good](../examples/Valid.png)  Hysteresis Fixed

|Expected|Actual|
|-|-|
|`['StartSection Name: A', 'EndSection Name: A']`|`['StartSection Name: A', 'EndSection Name: A']`|
|`['StartSection Name: B', 'EndSection Name: B']`|`['StartSection Name: B', 'EndSection Name: B']`|


In [26]:
pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A', 'EndSection Name: A'],
 ['StartSection Name: B', 'EndSection Name: B']]


![Good](../examples/Valid.png)  Hysteresis Fixed for more repetitions.
|Expected|Actual|
|-|-|
|`['StartSection Name: A', 'EndSection Name: A']`|`['StartSection Name: A', 'EndSection Name: A']`|
|`['StartSection Name: B', 'EndSection Name: B']`|`['StartSection Name: B', 'EndSection Name: B']`|


In [27]:
full_section = Section(
    section_name='Full',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=sub_section
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A', 'EndSection Name: A'],
 ['StartSection Name: B', 'EndSection Name: B']]


![Good](../examples/Valid.png)  Hysteresis Fixed Redefining `full_section`
|Expected|Actual|
|-|-|
|`['StartSection Name: A', 'EndSection Name: A']`|`['StartSection Name: A', 'EndSection Name: A']`|
|`['StartSection Name: B', 'EndSection Name: B']`|`['StartSection Name: B', 'EndSection Name: B']`|


In [28]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After')
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A', 'EndSection Name: A'],
 ['StartSection Name: B', 'EndSection Name: B']]


![Good](../examples/Valid.png)  Hysteresis Fixed Redefining `sub_section`.
|Expected|Actual|
|-|-|
|`['StartSection Name: A', 'EndSection Name: A']`|`['StartSection Name: A', 'EndSection Name: A']`|
|`['StartSection Name: B', 'EndSection Name: B']`|`['StartSection Name: B', 'EndSection Name: B']`|


In [29]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After')
    )

full_section = Section(
    section_name='Full',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=sub_section
    )

pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A', 'EndSection Name: A'],
 ['StartSection Name: B', 'EndSection Name: B']]


![Good](../examples/Valid.png) 
Redefining __Both__ `sub_section` and `full_section` fixes the issue.

|Expected|Actual|
|-|-|
|`['StartSection Name: A', 'EndSection Name: A']`|`['StartSection Name: A', 'EndSection Name: A']`|
|`['StartSection Name: B', 'EndSection Name: B']`|`['StartSection Name: B', 'EndSection Name: B']`|

### Try resetting the source

- Clear Hysteresis by re-defining both `sub_section` and `full_section`

In [30]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After')
    )

full_section = Section(
    section_name='Full',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=sub_section
    )

In [31]:
pprint(full_section.read(GENERIC_TEST_TEXT, context={'Dummy': 'Blank1'}))

full_section.source = None
sub_section.source = None

pprint(full_section.read(GENERIC_TEST_TEXT, context={'Dummy': 'Blank1'}))

[['StartSection Name: A', 'EndSection Name: A'],
 ['StartSection Name: B', 'EndSection Name: B']]
[['StartSection Name: A', 'EndSection Name: A'],
 ['StartSection Name: B', 'EndSection Name: B']]


- Resetting the source does not solve the problem.
- First time it runs as expected
- The second time, a line in the first subsection is missing.

|Expected|First Run (`GENERIC_TEST_TEXT1`)|Second Run (`GENERIC_TEST_TEXT2`)|
|-|-|-|
||![Good](../examples/Valid.png) Both full subsections found. No empty lists afterwards.|![Good](../examples/Valid.png)  Hysteresis Fixed|
|`['StartSection Name: A', 'EndSection Name: A']`|`['StartSection Name: A', 'EndSection Name: A']`|`['StartSection Name: A', 'EndSection Name: A']`|
|`['StartSection Name: B', 'EndSection Name: B']`|`['StartSection Name: B', 'EndSection Name: B']`|`['StartSection Name: B', 'EndSection Name: B']`|


### Use two different text sources

In [32]:
GENERIC_TEST_TEXT1 = [
    'Text to be ignored #1',
    'StartSection Name: A',
    'EndSection Name: A',
    'StartSection Name: B',
    'EndSection Name: B',
    'More text to be ignored #1',
    ]

In [33]:
GENERIC_TEST_TEXT2 = [
    'Text to be ignored #2',
    'StartSection Name: C',
    'EndSection Name: C',
    'StartSection Name: D',
    'EndSection Name: D',
    'More text to be ignored #2',
    ]

- Clear Hysteresis by re-defining both `sub_section` and `full_section`

In [34]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After')
    )

full_section = Section(
    section_name='Full',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=sub_section
    )

- First run with `GENERIC_TEST_TEXT1`
- Second run with `GENERIC_TEST_TEXT2`

In [35]:
pprint(full_section.read(GENERIC_TEST_TEXT1))
pprint(full_section.read(GENERIC_TEST_TEXT2))

[['StartSection Name: A', 'EndSection Name: A'],
 ['StartSection Name: B', 'EndSection Name: B']]
[['StartSection Name: C', 'EndSection Name: C'],
 ['StartSection Name: D', 'EndSection Name: D']]


- First time it runs as expected
- The second time, a line in the first subsection is missing.
  


|Expected|First Run (`GENERIC_TEST_TEXT1`)|Second Run (`GENERIC_TEST_TEXT2`)|
|-|-|-|
||![Good](../examples/Valid.png) Both full subsections found. No empty lists afterwards.|![Good](../examples/Valid.png)  Hysteresis Fixed|
|`['StartSection Name: A', 'EndSection Name: A']`(or `C`)|`['StartSection Name: A', 'EndSection Name: A']`|`['StartSection Name: C', 'EndSection Name: C']`|
|`['StartSection Name: B', 'EndSection Name: B']`(or `D`)|`['StartSection Name: B', 'EndSection Name: B']`|`['StartSection Name: D', 'EndSection Name: D']`|



### Compare *Source* before and after
- Compare `full_section` and `sub_section` ___Source___ after first and second run.

#### First Run

In [36]:
pprint(full_section.read(GENERIC_TEST_TEXT, context={'Dummy': 'Blank1'}))
print()
print('full_section source after first run')
print(repr(full_section.source))
print()
print('sub_section source after first run')
print(repr(sub_section.source))

[['StartSection Name: A', 'EndSection Name: A'],
 ['StartSection Name: B', 'EndSection Name: B']]

full_section source after first run
BufferedIterator(source=<list_iterator object at 0x0000021B14AF0E20>, buffer_size=5)
	BufferedIterator.previous_items = deque(['StartSection Name: A', 'EndSection Name: A', 'StartSection Name: B', 'EndSection Name: B'], maxlen=5)
	BufferedIterator.future_items = deque(['More text to be ignored'], maxlen=5)
	BufferedIterator._step_back = 0

sub_section source after first run
BufferedIterator(source=<list_iterator object at 0x0000021B14AF0E20>, buffer_size=5)
	BufferedIterator.previous_items = deque(['StartSection Name: A', 'EndSection Name: A', 'StartSection Name: B', 'EndSection Name: B'], maxlen=5)
	BufferedIterator.future_items = deque(['More text to be ignored'], maxlen=5)
	BufferedIterator._step_back = 0


#### Second Run

In [37]:
pprint(full_section.read(GENERIC_TEST_TEXT, context={'Dummy': 'Blank1'}))
print()
print('full_section source after first run')
print(repr(full_section.source))
print()
print('sub_section source after first run')
print(repr(sub_section.source))

[['StartSection Name: A', 'EndSection Name: A'],
 ['StartSection Name: B', 'EndSection Name: B']]

full_section source after first run
BufferedIterator(source=<list_iterator object at 0x0000021B14AF0520>, buffer_size=5)
	BufferedIterator.previous_items = deque(['StartSection Name: A', 'EndSection Name: A', 'StartSection Name: B', 'EndSection Name: B'], maxlen=5)
	BufferedIterator.future_items = deque(['More text to be ignored'], maxlen=5)
	BufferedIterator._step_back = 0

sub_section source after first run
BufferedIterator(source=<list_iterator object at 0x0000021B14AF0520>, buffer_size=5)
	BufferedIterator.previous_items = deque(['StartSection Name: A', 'EndSection Name: A', 'StartSection Name: B', 'EndSection Name: B'], maxlen=5)
	BufferedIterator.future_items = deque(['More text to be ignored'], maxlen=5)
	BufferedIterator._step_back = 0


- Source is identical for main section and subsection.
- Source is identical for between first and second run
- `future_items` contains *'More text to be ignored'*.

### Compare Context before and after
- Compare `full_section` and `sub_section` ___Context___ after first and second run.

- Clear Hysteresis by re-defining both `sub_section` and `full_section`

In [38]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After')
    )

full_section = Section(
    section_name='Full',
    end_section=SectionBreak('ignored', break_offset='Before'),
    subsections=sub_section
    )

#### First Run

In [39]:
pprint(full_section.read(GENERIC_TEST_TEXT, context={'Dummy': 'Blank1'}))
print()
print('full_section context after first run')
pprint(full_section.context)
pprint(full_section.scan_status)
print()
print('sub_section context after first run')
pprint(sub_section.context)
pprint(sub_section.scan_status)

[['StartSection Name: A', 'EndSection Name: A'],
 ['StartSection Name: B', 'EndSection Name: B']]

full_section context after first run
{'Break': 'SectionBreak',
 'Current Section': 'SubSection',
 'Dummy': 'Blank1',
 'Event': 'StartSection',
 'Skipped Lines': [],
 'Status': 'End of Source'}
'Break Triggered'

sub_section context after first run
{'Break': 'SectionBreak',
 'Current Section': 'SubSection',
 'Dummy': 'Blank1',
 'Event': 'StartSection',
 'Skipped Lines': [],
 'Status': 'End of Source'}
'Scan Complete'


- Context is identical for main section and subsection.
- `'Current Section': 'SubSection'` given for both.
- Initial 'Dummy' context item remains.
- `'Skipped Lines': []` consistent with *main section* 

#### Second Run

In [40]:
pprint(full_section.read(GENERIC_TEST_TEXT, context={'Dummy': 'Blank2'}))
print()
print('full_section context after second run')
pprint(full_section.context)
pprint(full_section.scan_status)
print()
print('sub_section context after second run')
pprint(sub_section.context)
pprint(sub_section.scan_status)

[['StartSection Name: A', 'EndSection Name: A'],
 ['StartSection Name: B', 'EndSection Name: B']]

full_section context after second run
{'Break': 'SectionBreak',
 'Current Section': 'SubSection',
 'Dummy': 'Blank2',
 'Event': 'StartSection',
 'Skipped Lines': [],
 'Status': 'End of Source'}
'Break Triggered'

sub_section context after second run
{'Break': 'SectionBreak',
 'Current Section': 'SubSection',
 'Dummy': 'Blank2',
 'Event': 'StartSection',
 'Skipped Lines': [],
 'Status': 'End of Source'}
'Scan Complete'


- Context is identical for *full_section* and *sub_section*.
- *Skipped Lines* contains ___['EndSection Name: A']___
- `'Current Section': 'SubSection'` given for both.
- 'Dummy' context item has correctly become *Blank2*.
- Context is identical for main section and subsection.
- `'Current Section': 'SubSection'` given for both. 
- `full_section.scan_status` is *Break Triggered*, `sub_section.scan_status` is *Scan Complete*


**full_section Context**
<table>
    <thead>
        <th>Key</th>
        <th>Expected Value</th>
        <th>First Run Value</th>
        <th>Second Run Value</th>
        <th>Test</th>
    </thead>
        <tr><td>Current Section</td>
            <td>Full</td>
            <td>SubSection</td>
            <td>SubSection</td>
            <td><img src="../examples/error.png" alt="Bad"/></td></tr>
    <tr><td>Dummy</td>
        <td>Blank#</td>
        <td>Blank1</td>  
        <td>Blank2</td>
        <td><img src="../examples/Valid.png" alt="Good"/></td></tr>
    <tr><td>Skipped Lines</td>  
        <td>[]</td>
        <td>[]</td>            
        <td>['EndSection Name: A']</td>
        <td><img src="../examples/error.png" alt="Bad"/></td></tr>
    <tr><td>Status</td>
        <td><B><U>?</B></U></td>
        <td>End of Source</td> 
        <td>End of Source</td>
        <td><B><U>?</B></U></td></tr>
    <tr><td>Break</td>          
        <td>SectionBreak</td>
        <td>SectionBreak</td>  
        <td>SectionBreak</td>
        <td><img src="../examples/Valid.png" alt="Good"/></td></tr>
    <tr><td>Event</td>          
        <td>ignored</td>
        <td>StartSection</td>  
        <td>StartSection</td>
        <td><img src="../examples/error.png" alt="Bad"/></td></tr>
</table>


**sub_section Context**
<table>
    <thead>
        <th>Key</th>
        <th>Expected Value</th>
        <th>First Run Value</th>
        <th>Second Run Value</th>
        <th>Test</th>
    </thead>
        <tr><td>Current Section</td>
            <td>SubSection</td>
            <td>SubSection</td>
            <td>SubSection</td>
            <td><img src="../examples/Valid.png" alt="Good"/></td></tr>
    <tr><td>Dummy</td>
        <td>Blank#</td>
        <td>Blank1</td>  
        <td>Blank2</td>
        <td><img src="../examples/Valid.png" alt="Good"/></td></tr>
    <tr><td>Skipped Lines</td>  
        <td>['EndSection Name: A']</td>
        <td>[]</td>            
        <td>['EndSection Name: A']</td>
        <td><img src="../examples/error.png" alt="Bad"/></td></tr>
    <tr><td>Status</td>
        <td><B><U>?</B></U></td>
        <td>End of Source</td> 
        <td>End of Source</td>
        <td><B><U>?</B></U></td></tr>
    <tr><td>Break</td>          
        <td>SectionBreak</td>
        <td>SectionBreak</td>  
        <td>SectionBreak</td>
        <td><img src="../examples/Valid.png" alt="Good"/></td></tr>
    <tr><td>Event</td>          
        <td>EndSection</td>
        <td>StartSection</td>  
        <td>StartSection</td>
        <td><img src="../examples/error.png" alt="Bad"/></td></tr>
</table>


# SectionBreak._count_down is not reset between runs
FIXME Reset SectionBreak on Section Start

**full_section Context**
<table>
    <thead>
        <th>Key</th>
        <th>Expected Value</th>
        <th>First Run Value</th>
        <th>Second Run Value</th>
        <th>Test</th>
    </thead>
        <tr><td>Current Section</td>
            <td>Full</td>
            <td>SubSection</td>
            <td>SubSection</td>
            <td><img src="../examples/error.png" alt="Bad"/></td></tr>
    <tr><td>Dummy</td>
        <td>Blank#</td>
        <td>Blank1</td>  
        <td>Blank2</td>
        <td><img src="../examples/Valid.png" alt="Good"/></td></tr>
    <tr><td>Skipped Lines</td>  
        <td>[]</td>
        <td>[]</td>            
        <td>['EndSection Name: A']</td>
        <td><img src="../examples/error.png" alt="Bad"/></td></tr>
    <tr><td>Status</td>
        <td><B><U>?</B></U></td>
        <td>End of Source</td> 
        <td>End of Source</td>
        <td><B><U>?</B></U></td></tr>
    <tr><td>Break</td>          
        <td>SectionBreak</td>
        <td>SectionBreak</td>  
        <td>SectionBreak</td>
        <td><img src="../examples/Valid.png" alt="Good"/></td></tr>
    <tr><td>Event</td>          
        <td>ignored</td>
        <td>StartSection</td>  
        <td>StartSection</td>
        <td><img src="../examples/error.png" alt="Bad"/></td></tr>
</table>


# Context Fixes to do
1. Create ProtectedDict subclass of dict
   - Add optional protected_items to __ init __
   - Redefine update to prevent updating items with keys in the protected_items list.
2. convert self.scan_status to a property that sets and gets from self.context['Status']
3. convert self.section_name to a property that sets and gets from self.context['Current Section']
4. in `step_source` remove `break_context` and use `self.context['Status']`

In [41]:
# FIXME Context updates should not be quite so indiscriminate
#     self.context.update(sub_rdr.context)
#     self.context.update(break_context)
# TODO Setup a Local Context, which does not get passed to parent sections
# TODO self.scan_status should become a private variable which sets and gets from self.context['Status']
# TODO self.section_name should become a private variable which sets and gets from self.context['Current Section']
# TODO in step_source remove break_context and use self.context['Status']


***Context Standard Items***
|Item Key|Value Description|
|-|-|
|Current Section|The assigned name of the section `self.section_name`|
|Skipped Lines|A list of items in the source occurring before the first item of the section|
|Status|The current section scanning status one of 'Not Started', 'RuntimeError', 'End of Source'. 'Scan In Progress'|
|Break|The name assigned to the SectionBreak that most recently triggered. Before the section start this is not defined. While the section scan is in progress, this is the name of the StartSection. After the section ends, this is the name of the the EndSection.
|Event|The result from the SectionBreak that most recently triggered; the `TriggerEvent.test_value`.  For more information see the TriggerEvent documentation.|
                

**********

# DONE TO HERE

**********

### Initial Section and Sub-Section Definitions

- Only definition line is:<br>
`subsections=[sub_section]`

In [42]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['Text to be ignored',
  'StartSection Name: A',
  'EndSection Name: A',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Good](../examples/Valid.png) Results in all lines in one sub-list

**********

## Section Break options

### Add start to Section Definition

- Section start **Before** *StartSection*

`start_section=SectionBreak('StartSection', break_offset='Before'),`

In [43]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A',
  'EndSection Name: A',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Good](../examples/Valid.png) Includes all lines after first *StartSection* in single sub-list.

### Add end to Section Definition

- Section start **Before** *StartSection*
- Section end **After** *EndSection*

```python
full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
```

In [44]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A', 'EndSection Name: A']]


- ![Good](../examples/Valid.png) Includes both lines of first section in single sub-list.

### Define Muti-Section to Read Both Sections

Add end to Section Definition
- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- Simple subsection
- Multi Section defines Full Section as Sub Section with no start or end (All lines)

```python
sub_section = Section(section_name='SubSection')

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )

multi_section = Section(
    section_name='Multi',
    subsections=[full_section] 
    )
```

In [45]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )

multi_section = Section(
    section_name='Multi',
    subsections=[full_section] 
    )

pprint(multi_section.read(GENERIC_TEST_TEXT))

[[['StartSection Name: A', 'EndSection Name: A']],
 [['StartSection Name: B', 'EndSection Name: B']],
 [[]]]


- ![Bad](../examples/error.png) Includes both lines of each section in its own sub-list, but adds a blank sub-list at the end.
- ??? Is this incorrect or is this expected???

### Set Same Start and End Breaks for Section

Add end to Section Definition
- Section start **Before** *StartSection*
- Section end **After** *StartSection*
- Simple subsection
- Multi Section defines Full Section as Sub Section with no start or end (All lines)

```python
sub_section = Section(section_name='SubSection')

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before'),
    subsections=[sub_section] 
    )

multi_section = Section(
    section_name='Multi',
    subsections=[full_section] 
    )
```

In [46]:
full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before')
    )

multi_section = Section(section_name='Multi',
    subsections=[full_section] 
    )

pprint(multi_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A', 'EndSection Name: A'],
 ['StartSection Name: B', 'EndSection Name: B', 'More text to be ignored']]


- Without SubSection, it works as expected.

- ![Bad](../examples/error.png) Still Appears to hang

**********

## SubSection Break Options

### Add *End on First* to Sub-Section

- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- SubSection **End On First**

```python
sub_section = Section(section_name='SubSection',
    end_on_first_item=True,
    )

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
```

In [47]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='After')
    end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )

pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A', 'EndSection Name: A']]


- ![Good](../examples/Valid.png) Results in `[['StartSection Name: A', 'EndSection Name: A']]`.

### Add *End* as **True** to Sub-Section

- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- SubSection end is **True**  Should be the same as **End On First**

```python
sub_section = Section(section_name='SubSection',
    end_section=SectionBreak(True)
    )

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
```

In [48]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='After')
    #end_on_first_item=True,
    #keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )

pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A'], []]


## What should  `end_section=SectionBreak(True)` result in?

- ![Bad](../examples/error.png) Results in `['StartSection Name: A'], []` sub-lists, should have been `['StartSection Name: A']`.
- ??? Is this incorrect or is this expected???

### Add *Start* to Sub-Section

- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- SubSection start **Before** *EndSection*

```python
sub_section = Section(section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before')
    )

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
```

In [49]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before')
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )

pprint(full_section.read(GENERIC_TEST_TEXT))

[[]]


- ![Bad](../examples/error.png) Results in blank sub-list.

### Change Start Sub-Section to **After**

- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- SubSection start **Before** *EndSection*

```python
sub_section = Section(section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before')
    )

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
```

In [50]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='After')
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )

pprint(full_section.read(GENERIC_TEST_TEXT))

[[]]


- ![Bad](../examples/error.png) Results in blank sub-list.

### Add start to SubSection Definition
> `start_section=SectionBreak('EndSection', break_offset='Before')`

In [51]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Bad](../examples/Error.png) - ??? Is this incorrect or is this expected???

### Add *Start After* to Section Definition
> `start_section=SectionBreak('StartSection', break_offset='After')`

In [52]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Good](../examples/Valid.png) Results in one line section

### Add *Start After StartSection* to Section Definition and *Start Before End Section* to SubSection Definition
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After')`
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before')`
    

In [53]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Bad](../examples/error.png) - ??? Is this incorrect or is this expected???

### Add *Start After StartSection* to Section Definition and *Start Before End Section* to SubSection Definition and set *End On First Line* for SubSection
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After')`
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`
> `end_on_first_item=True,`
    

In [54]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Bad](../examples/Error.png) - ??? Is this incorrect or is this expected???

### Add *Start After StartSection* to Section Definition, and for SubSection Definition, set *Start* to  *Before EndSection* and *End* to *`True` (Always Break)*
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After')`
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`
> `end_section=SectionBreak(True),`
    

In [55]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A'], ['EndSection Name: B'], []]


- ![Good](../examples/Valid.png) Results in one line section

### Add *Start After StartSection* and *End Before EndSection* to Section Definition, and for SubSection Definition, set *Start* to  *Before EndSection* and *End* to *`True` (Always Break)*
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After'),`
> `end_section=SectionBreak('EndSection', break_offset='After'),`
    
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`
> `end_section=SectionBreak(True),`
    

In [56]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A']]


- ![Good](../examples/Valid.png) Results in one line section

### Add *Start __Before__ StartSection* and *End Before EndSection* to Section Definition, and for SubSection Definition, set *Start* to  *Before EndSection* and *End* to *`True` (Always Break)*
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='Before'),`
> `end_section=SectionBreak('EndSection', break_offset='After'),`
    
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`
> `end_section=SectionBreak(True),`
    

In [57]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[[]]


- ![Bad](../examples/Error.png) Results in empty list of lists
- ??? Is this incorrect or is this expected???

### Add *Start __Before__ StartSection* and *End Before EndSection* to Section Definition, and for SubSection Definition, set *Start* to *Before EndSection*, *End* to *`True` (Always Break)* and *Keep Partial* to *`True`*
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='Before'),`
> `end_section=SectionBreak('EndSection', break_offset='After'),`    
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`<br>
> `end_section=SectionBreak(True),`<br>
> `keep_partial=True,`
    

In [58]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[[]]


- ![Bad](../examples/Error.png) Results in empty list of lists
- ??? Is this incorrect or is this expected???

### Add *Start __Before__ StartSection* and *End Before EndSection* to Section Definition, and don't set any SectionBreaks for SubSection Definition, 
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='Before'),`
> `end_section=SectionBreak('EndSection', break_offset='After'),`    
    

In [59]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A', 'EndSection Name: A']]


- ![Good](../examples/Valid.png) Results in Two line section

### Add *Start __After__ StartSection* and *End Before EndSection* to Section Definition, and don't set any SectionBreaks for SubSection Definition, 
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After'),`
> `end_section=SectionBreak('EndSection', break_offset='After'),`    
    

In [60]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A']]


- ![Good](../examples/Valid.png) Results in one line section